In [13]:
import pandas as pd


In [14]:
df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla.csv")

In [15]:
df.shape[0]

8322

In [16]:
lai = 0.15
tc = 5

df.columns = map(str.lower, df.columns)
if 'fecha' in df.columns:
    df = df.drop('fecha', axis=1)

# change column name 'velocidad'
substrings_to_replace_velocidad = ["vv", "viento", "velocidad", "velocidad del viento"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_velocidad):
        new_column_name = "Velocidad del viento (m/s)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'mp2,5'
substrings_to_replace_mp = ["mp", "2,5", "25"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "MP2,5 (µg/m³)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'altura'
substrings_to_replace_mp = ["mh", "altura", "mezcla"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "Altura de Mezcla (m)"
        df.rename(columns={column: new_column_name}, inplace=True)

df['Velocidad del viento (m/s)'] = df['Velocidad del viento (m/s)'].apply(lambda x: float(str(x).replace(",","."))).round()

table1 = {
    'Velocidad del viento (m/s)': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    'Promedio': [0, 0.03, 0.09, 0.15, 0.17, 0.19, 0.2, 0.56, 0.92, 0.92, 2.11, 2.11, 2.11, 2.11],
    'Mínimo': [0, 0.006, 0.012, 0.018, 0.022, 0.025, 0.029, 0.056, 0.082, 0.082, 0.57, 0.57, 0.57, 0.57],
    'Máximo': [0, 0.042, 0.163, 0.285, 0.349, 0.414, 0.478, 1.506, 2.534, 2.534, 2.534, 2.534, 2.534, 2.534],
    '% Resuspensión': [0, 1.5, 3, 4.5, 6, 7.5, 9, 10, 11, 12, 13, 16, 20, 23]
}
reference_table = pd.DataFrame(table1)

df = pd.merge(df, reference_table, on='Velocidad del viento (m/s)', how='left')

df['LAI (m2/m2)'] = lai

df['Vd,min (cm/s)'] = df['LAI (m2/m2)'] * df['Mínimo']
df['Vd,max (cm/s)'] = df['LAI (m2/m2)'] * df['Máximo']
df['Vd (cm/s)'] = df['LAI (m2/m2)'] * df['Promedio']

flux = pd.DataFrame()
flux['MP2,5 (g/m³)']    = df['MP2,5 (µg/m³)'] / 1000 / 1000
flux["Vd,min (m/s)"]    = df['Vd,min (cm/s)'] / 100
flux["Vd,max (m/s)"]    = df['Vd,max (cm/s)'] / 100
flux["Vd (m/s)"]        = df['Vd (cm/s)'] / 100
flux["fmin,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,min (m/s)"] * 3600
flux["fmax,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,max (m/s)"] * 3600
flux["ft (g/m2*h)"]     = flux['MP2,5 (g/m³)'] * flux["Vd (m/s)"] * 3600

In [113]:
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.16f}'.format

In [17]:
resuspension = pd.DataFrame()
resuspension['% Resuspensión']  = df['% Resuspensión']
resuspension['fmin,t (g/m2*h)'] = flux["fmin,t (g/m2*h)"]
resuspension['fmax,t (g/m2*h)'] = flux["fmax,t (g/m2*h)"]
resuspension['ft (g/m2*h)']     = flux["ft (g/m2*h)"]
resuspension['At min (g/m2*h)'] = 0
resuspension['At max (g/m2*h)'] = 0
resuspension['At (g/m2*h)']     = 0
resuspension['Rmin (g/m2*h)']   = 0
resuspension['Rmax (g/m2*h)']   = 0
resuspension['Rt (g/m2*h)']     = 0

# rows = []
# # for fila in range(resuspension.shape[0]-1):
# for fila in range(2):

#     resuspension['Rmin (g/m2*h)']   = ( resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
#     resuspension['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] - resuspension['Rmin (g/m2*h)']
#     resuspension['Rmax (g/m2*h)']   = ( resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
#     resuspension['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] - resuspension['Rmax (g/m2*h)']
#     resuspension['Rt (g/m2*h)']     = ( resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
#     resuspension['At (g/m2*h)']     = resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] - resuspension['Rt (g/m2*h)']

#     print(f"""
#         fila = {fila}
#         resuspension['Rmin (g/m2*h)']   = ( {resuspension['At min (g/m2*h)'].shift(1)} + {resuspension['fmin,t (g/m2*h)'].iloc[-1]} ) * {resuspension['% Resuspensión']} / 100
#         resuspension['At min (g/m2*h)'] =   {resuspension['At min (g/m2*h)'].shift(1)} + {resuspension['fmin,t (g/m2*h)']}   - {resuspension['Rmin (g/m2*h)']}
#         resuspension['Rmax (g/m2*h)']   = ( {resuspension['At max (g/m2*h)'].shift(1)} + {resuspension['fmax,t (g/m2*h)']} ) * {resuspension['% Resuspensión']} / 100
#         resuspension['At max (g/m2*h)'] =   {resuspension['At max (g/m2*h)'].shift(1)} + {resuspension['fmax,t (g/m2*h)']}   - {resuspension['Rmax (g/m2*h)']}
#         resuspension['Rt (g/m2*h)']     = ( {resuspension['At (g/m2*h)'].shift(1)}     + {resuspension['ft (g/m2*h)']}     ) * {resuspension['% Resuspensión']} / 100
#         resuspension['At (g/m2*h)']     =   {resuspension['At (g/m2*h)'].shift(1)}     + {resuspension['ft (g/m2*h)']}       - {resuspension['Rt (g/m2*h)']}
#     """)

#     row = {}
#     row['Rmin (g/m2*h)']   = resuspension['Rmin (g/m2*h)'].iloc[fila+1]
#     row['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].iloc[fila+1]
#     row['Rmax (g/m2*h)']   = resuspension['Rmax (g/m2*h)'].iloc[fila+1]
#     row['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].iloc[fila+1]
#     row['Rt (g/m2*h)']     = resuspension['Rt (g/m2*h)'].iloc[fila+1]
#     row['At (g/m2*h)']     = resuspension['At (g/m2*h)'].iloc[fila+1]

#     rows.append(row)

In [32]:
user_input = 3
A = [5,4,4,2,3]
B = [3,1,1,5,3]

# Initialize the sum
C = user_input
D = 0
result = 0

# Calculate the sum dynamically based on the pattern
for a,b in zip(A,B):
    D = C * b + a
    print(f'{D} = {C} * {b} + {a}')
    C = C + D
    print(C)
    result += D

print(result)


14 = 3 * 3 + 5
17
21 = 17 * 1 + 4
38
42 = 38 * 1 + 4
80
402 = 80 * 5 + 2
482
1449 = 482 * 3 + 3
1931
1928


In [56]:
resuspension

,% Resuspensión,"fmin,t (g/m2*h)","fmax,t (g/m2*h)",ft (g/m2*h),At min (g/m2*h),At max (g/m2*h),At (g/m2*h),Rmin (g/m2*h),Rmax (g/m2*h),Rt (g/m2*h)
0,4.5,2.488320e-06,3.939840e-05,2.073600e-05,0,0,0,0,0,0
1,4.5,6.804000e-07,1.077300e-05,5.670000e-06,0,0,0,0,0,0
2,3.0,3.499200e-07,4.753080e-06,2.624400e-06,0,0,0,0,0,0
3,1.5,1.296000e-07,9.072000e-07,6.480000e-07,0,0,0,0,0,0
4,1.5,3.726000e-07,2.608200e-06,1.863000e-06,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8317,6.0,1.199880e-06,1.903446e-05,9.271800e-06,0,0,0,0,0,0
8318,4.5,1.146960e-06,1.816020e-05,9.558000e-06,0,0,0,0,0,0
8319,4.5,1.215000e-06,1.923750e-05,1.012500e-05,0,0,0,0,0,0
8320,3.0,6.998400e-07,9.506160e-06,5.248800e-06,0,0,0,0,0,0


In [116]:
columns = zip(
    resuspension['% Resuspensión'][1:],
    resuspension['fmin,t (g/m2*h)'][1:]
)

In [117]:
At_min = 0

for Resuspensión,fmin in columns:
    Rmin = ( At_min + fmin ) * Resuspensión / 100
    print(f'Rmin: {Rmin} = ({At_min} + {fmin}) * {Resuspensión} / 100')
    At_min = At_min + fmin - Rmin
    print(At_min, "\n")

print(At_min, Rmin)


Rmin: 3.061799999999999e-08 = (0 + 6.803999999999999e-07) * 4.5 / 100
6.497819999999998e-07 

Rmin: 2.999105999999999e-08 = (6.497819999999998e-07 + 3.4992e-07) * 3.0 / 100
9.697109399999999e-07 

Rmin: 1.6489664099999998e-08 = (9.697109399999999e-07 + 1.296e-07) * 1.5 / 100
1.0828212758999998e-06 

Rmin: 2.1831319138499996e-08 = (1.0828212758999998e-06 + 3.7260000000000004e-07) * 1.5 / 100
1.4335899567614999e-06 

Rmin: 2.72872493514225e-08 = (1.4335899567614999e-06 + 3.8556e-07) * 1.5 / 100
1.7918627074100775e-06 

Rmin: 0.0 = (1.7918627074100775e-06 + 0.0) * 0.0 / 100
1.7918627074100775e-06 

Rmin: 3.4070740611151163e-08 = (1.7918627074100775e-06 + 4.7952e-07) * 1.5 / 100
2.2373119667989263e-06 

Rmin: 1.067769590039678e-07 = (2.2373119667989263e-06 + 1.32192e-06) * 3.0 / 100
3.4524550077949586e-06 

Rmin: 2.642730753507731e-07 = (3.4524550077949586e-06 + 2.4202799999999994e-06) * 4.5 / 100
5.608461932444184e-06 

Rmin: 4.40576515946651e-07 = (5.608461932444184e-06 + 1.7344799999999

In [121]:
a = 0.0001 + 0.0002
a

0.00030000000000000003

In [123]:
b = 1 + 2
b / 10000

0.0003